In [1]:
import os
import preprocess_data as prep

### Q1. Install MLflow

In [ ]:
!conda install mlflow -y

In [1]:
import mlflow

In [8]:
!mlflow --version

mlflow, version 2.12.2


What's the version that you have? **2.12.2**

### Q2. Download and preprocess the data

In [ ]:
!wget https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/refs/heads/main/cohorts/2025/02-experiment-tracking/homework/preprocess_data.py

In [ ]:
links = ["https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet",
"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet",
"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet"]

for link in links:
    os.system(f"wget {link} -P taxi_data")

In [8]:
os.system(f"python preprocess_data.py --raw_data_path taxi_data --dest_path ./output")

0

How many files were saved to OUTPUT_FOLDER? **4**

### Q3. Train a model with autolog

In [ ]:
train_link = "https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/refs/heads/main/cohorts/2025/02-experiment-tracking/homework/train.py"
os.system(f"wget {train_link}")

In [12]:
os.system(f"python train.py --data_path ./output")

2025/05/28 07:19:11 INFO mlflow.tracking.fluent: Experiment with name 'experiment_tracking_homework' does not exist. Creating a new experiment.
2025/05/28 07:19:12 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ee1822d0d37a441faefc2d2d56f0672a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


0

What is the value of the min_samples_split parameter: **2**

### Q4. Launch the tracking server locally

In addition to backend-store-uri, what else do you need to pass to properly configure the server? **default-artifact-root**

### Q5. Tune model hyperparameters

In [2]:
tune_link = "https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/refs/heads/main/cohorts/2025/02-experiment-tracking/homework/hpo.py"
os.system(f"wget {tune_link}")

--2025-05-29 21:09:31--  https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/refs/heads/main/cohorts/2025/02-experiment-tracking/homework/hpo.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1836 (1.8K) [text/plain]
Saving to: ‘hpo.py’

     0K .                                                     100% 32.4M=0s

2025-05-29 21:09:31 (32.4 MB/s) - ‘hpo.py’ saved [1836/1836]



0

In [6]:
os.system(f"python hpo.py")

2025/05/29 21:30:02 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.


🏃 View run brawny-shrimp-840 at: http://127.0.0.1:5000/#/experiments/1/runs/0dd74398a40f4cc993daf10d6a7b3cc1

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1

🏃 View run lyrical-bass-586 at: http://127.0.0.1:5000/#/experiments/1/runs/361e5464fa0f43dc9bf6fa88e14722e3

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

🏃 View run able-gnu-521 at: http://127.0.0.1:5000/#/experiments/1/runs/db2d20aa86dc439f99dc68ea493dc0ea

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run smiling-stoat-430 at: http://127.0.0.1:5000/#/experiments/1/runs/059bd8ff4aa74e79a3ea9e1f481064d4

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run thoughtful-hog-174 at: http://127.0.0.1:5000/#/experiments/1/runs/cc49ddd9bd0242818aba98f11ed8ed92

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                     

🏃 View run melodic-lark-857 at: http://127.0.0.1:5000/#/experiments

0

What is the test RMSE of the best model? **5.0375**

### Q6. Promote the best model to the model registry

In [7]:
reg_link = "https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/refs/heads/main/cohorts/2025/02-experiment-tracking/homework/register_model.py"
os.system(f"wget {reg_link}")

--2025-05-29 22:54:14--  https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/refs/heads/main/cohorts/2025/02-experiment-tracking/homework/register_model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2487 (2.4K) [text/plain]
Saving to: ‘register_model.py’

     0K ..                                                    100% 18.9M=0s

2025-05-29 22:54:14 (18.9 MB/s) - ‘register_model.py’ saved [2487/2487]



0

In [ ]:
os.system(f"python register_model.py")